# Programowanie obiektowe
> I thought of objects being like biological cells and/or individual computers on a network, only able to communicate with messages (so messaging came at the very beginning – it took a while to see how to do messaging in a programming language efficiently enough to be useful).
> 
> Alan Kay - twórca paradygmatu obiektowego

## Klasy w Pythonie

In [ ]:
import yfinance as yf

class Stock:
    def __init__(self, ticker, name):
        self.ticker = ticker
        self._company_name = name

    @property
    def current_price(self):
        yf_stock = yf.Ticker(self.ticker)
        return self._get_current_price() or self._get_regular_market_price()

    def get_company_name(self):
        return self._company_name

    @staticmethod
    def from_ticker2(ticker):
        name = yf.Ticker(ticker).info.get("shortName")
        return Stock(ticker, name)
    
    @classmethod
    def from_ticker(cls, ticker):
        name = yf.Ticker(ticker).info.get("shortName")
        return cls(ticker, name)

    def _get_regular_market_price(self):
        return yf.Ticker(self.ticker).info["regularMarketPrice"]

    def _get_current_price(self):
        return yf.Ticker(self.ticker).info["currentPrice"]



nvda = Stock.from_ticker("NVDA")
nvda.from_ticker("MSFT")
print(f"Current price of {nvda.get_company_name()} is ${nvda.current_price:.2f}")
msft = Stock("MSFT", "Microsoft Corporation")
msft._get_current_price()

Powyżej widać przykład prostej klasy reprezentującej akcje przedsiębiorstw. Co ciekawsze jej fragmenty:
- `__init__` jest de facto tradycyjnie pojmowanym konstruktorem klasy, który działa już po zaalokowaniu pamięci na obiekt i służy do jego inicjalizacji
- wszystkie metody biorą specjalny parametr `self` reprezentujący obiekt, na którym wołana jest metoda
- `ticker` to pole publiczne
- `get_company_name` jest najzwyczajniejszą metodą publiczną (tj. będącą częścią kontraktu klasy) i zwraca wartość pola prywatnego
- `current_price` jest oznaczone jako property - w 4 od dołu linijce widać, że możemy używać tego jako pola, mimo, że jego wartość jest dynamicznie wyliczana
- `from_ticker` to metoda klasowa (w tym przypadku tzw. `factory method`), która na pierwszym parametrze bierze obiekt, reprezentujący klasę, na której została wywołana
- `_get_current_price` i `_get_regular_market_price` to metody prywatne (choć jak ktoś chce, to może je wywoływać poza tą klasą, bo Python tego nie zabrania - patrz ostatnia linijka - choć konwencja juz tak). Co do zasady ta konwencja pozwala zdefiniować stabilne publiczne API i API prywatne (prefiksowane `_`), którego stabilności nie gwarantuje się

## Podstawy dziedziczenia
Dziedziczenie klas to mechanizm pozwalający na realizację zależności, w której tzw. podklasa **jest** szczególnym przypadkiem klasy bazowej - np. akcja **jest** papierem wartościowym, ale np. akcja **ma** ticker. To ważne by odróżniać kiedy coś **jest** czymś, a kiedy coś **posiada** coś innego. Dziedziczenie stosuje się tylko gdy coś **jest** czymś, a nie po to by np. zaoszczędzic sobie pisania. Warto zachowywać zasady SOLID:
- single responsibility (tylko jeden powód do zmian)
- open/closed principle (otwartość na rozszerzanie - np. przez dziedziczenie i przedefiniowanie - oraz brak konieczności modyfikacji oryginalnej klasy)
- Liskov substitution principle - każdy egzemplarz podklasy musi pasować w każde miejsce oczekujące na obieekt klasy bazowej
- Interface segregation principle - kod zależący od danej klasy powinien zależeć tylko od potrzebnego mu interfejsu i nie powinien musieć zależeć od interfejsów mu niepotrzebnych
- Dependency inversion principle - klasy powinne zależeć od abstrakcyjnych interfejsów lub klas bazowych, a nie ich konkretnych implementacji

In [ ]:
class DataSource:
    def fetch(self, ticker, date_range):
        df = self.load_to_dataframe(ticker)
        df = self.standardize_dataframe(df)
        return self.trim_to_data_range(date_range, df)

    def load_to_dataframe(self, ticker):
        raise NotImplemented()

    def standardize_dataframe(self, df):
        df.dropna(inplace=True)
        df.sort_index(inplace=True)
        self._drop_unwanted_cols(df)
        return df

    def _drop_unwanted_cols(self, df):
        desired_cols = {"Open", "High", "Low", "Close", "Volume"}
        for col in df.columns:
            if col not in desired_cols:
                df.drop(col, axis=1, inplace=True)

    def trim_to_data_range(self, date_range, df):
        return df[(df.index >= date_range.start) & (df.index <= date_range.end)]


class LocalCSVDataSource(DataSource):
    def __init__(self: Self, data_folder: Path) -> None:
        self._data_folder = data_folder

    def load_to_dataframe(self, ticker) -> pd.DataFrame:
        try:
            file_name = f"{ticker.lower()}.us.txt"
            return pd.read_csv(
                self._data_folder / file_name, parse_dates=True, index_col=0
            )
        except FileNotFoundError as e:
            raise TickerNotFoundError() from e

## Wielodziedziczenie
Python w odróżnieniu od wielu innych nowoczesnych języków programowania wspiera wielodziedziczenie. Czym jest wielodziedziczenie? Mówiliśmy, że dziedziczenie modeluje relację `is-a`. W rzeczywistości zdarza się, że taka relacja zachodzi między różnymi bytami w zależności od poziomu abstrakcji na którym ją rozpatrujemy. Np.: samochód i rower są pojazdami, ale zarówno samochód jak i radiobudzik są urządzeniami mogącymi odtwarzać radio. W obu przypadkach relacja `is-a` zachodzi i w Pythonie możemy to explicite modelować:

In [ ]:
from typing import Self

class TradableSecurity:
    def buy(amount: int) -> int:
        ...

class EquityUnit:
    is_voting: bool
    share_ownership: int

class Stock(EquityUnit, TradableSecurity):
    def __init__(self: Self, ticker: str, price: int):
        self.ticker = ticker
        super(Stock, self).__init__(true, 1)
        

stock = Stock()
print(f"stock {'is' if stock.is_voting else 'is not'} voting")
stock.buy(10)
print(f"is Stock a TradableSecurity? {'yes' if isinstance(stock, TradableSecurity) else 'no'}")
print(f"is Stock a EquityUnit? {'yes' if isinstance(stock, EquityUnit) else 'no'}")

### super
`super` jest wbudowaną funkcją, z której możemy skorzystać by uzyskać dostęp do pól lub metod w wersjach zdefiniowanych w nadklasach - zwracany jest obiekt tymczasowy klasy bazowej (tak naprawdę tylko proxy), który może zostać "związany" z pożądaną metodą podczas jej wywołania. W swojej implementacji `super` korzysta z `mro` aby znaleźć właściwe wiązanie dla nazwy:

In [ ]:
class A:
    def do_something(self):
        print("I'm A")

class B(A):
    def do_something(self):
        print("I'm B")

class C(A):
    def do_something(self):
        print("I'm C")

class D(B, C):
    def do_something(self):
        print("I'm D")

    def do_something_else_D(self):
        super(D, self).do_something()

    def do_something_else_B(self):
        super(B, self).do_something()

    def do_something_else_C(self):
        super(C, self).do_something()
        
    def do_something_else_A(self):
        super(A, self).do_something()

d = D()
d.do_something()
d.do_something_else_D()
d.do_something_else_B()
d.do_something_else_C()
d.do_something_else_A()

`super()` bywa często używane gdy zachowanie override'owanej metody jest rozszerzeniem bardziej generalnego zachowania, zaimplementowanego już w klasie bazowej - szczególnie popularne jest wywoływanie konstruktora nadklasy celem inicjalizacji dziedziczonych pól.

In [ ]:
class AlphaVantageDataSource(DataSource):
    def __init__(self, api_key):
        self.__API_KEY = api_key

    def standardize_dataframe(self, df):
        df.columns = self._rename_columns(df)
        df.index = self._convert_index(df)
        super().standardize_dataframe(df)
        self._convert_columns(df)
        return df
    ...

In [ ]:
class Rectangle:
    def __init__(self, length, width):
        self.length = length
        self.width = width

    def area(self):
        return self.length * self.width

    def perimeter(self):
        return 2 * self.length + 2 * self.width

class Square(Rectangle):
    def __init__(self, length):
        super().__init__(length, length)


s = Square(5)
print(f"area: {s.area()}, perimeter: {s.perimeter()}")

Skomplikowane hierarchie (wielo-)dziedziczenia rzadko są dobrym rozwiązaniem problemów i w porządnie zaprojektowanych programach spotyka się je relatywnie rzadko lub wcale. Problemy takie jak nieoczywsta kolejność wyszukiwania nazw, ale też niepotrzebne wiązanie ze sobą niepowiązanych konceptów (to, że coś jest prawdą w rzeczywistości nie oznacza, że jest dobrym modelem danej domeny w kodzie) sprawiły, że od paru lat raczej unika się modelowania kodu z użyciem wielodziedziczenia. Niektóre nowoczesne języki programowania jak Go w ogóle zrezygnowały z dziedziczenia (nie tylko wielodziedziczenia), zachęcając programistów do poszukiwania innych sposobów strukturyzowania ich problemów. Również w Pythonie pojawiło się kilka innych rozwiązań, które pozwalają na tworzenie kodu łatwiejszego w utrzymaniu i dalszym rozwoju.

## Mixins - sensowne zastosowanie wielodziedziczenia

To oczywiście nie oznacza, że wielodziedziczenie odeszło w zapomnienie - wiele frameworków używa wielodziedziczenia w sposób zgoła odmienny od modelowania relacji is-a. Zamiast tego klasy użytkownika zazwyczaj dziedziczą po klasie bazowej z frameworka oferującej podstawową funkcjonalność danego konceptu, modelując rzeczywiście tę relację, ale pozostałe klasy bazowe służą zupelnie czemu innemu. Mowa o tzw. mixins - klasach bazowych, które enkapsulują pewne zachowanie, które ciężko jednoznacznie przypisać do danego typu. Przykładowo w popularnym frameworku webowym Django możemy stworzyć widok (jako klasę) i dodać w łatwy sposób weryfikacje czy użytkownik próbujacy go użyć jest zalogowany:

In [ ]:
class MyView(LoginRequiredMixin, View):
    login_url = '/login/'
    redirect_field_name = 'redirect_to'
    ... # normal View implementation

Podobnych użyć mixinów jest wiele - mogą np. dodawać do widoku walidacje formularzy, paginację list obiektów etc. Zdarza sie również, że mixiny są wykorzystywane również poza frameworkami - właśnie wówczas gdy chcemy zenkapsulować pewne zachowania, które nie są zależne od konkretnego typu, a co najwyżej od pewnych jego atrybutów, a które duplikują się między różnymi, niepowiązanymi klasami. Czasami są to po prostu zachowania, które możemy też opakować w dekorator, jednak z racji na konieczność przechowywania stanu lub wyższy poziom komplikacji czytelniej je zenkapsulować w formie klas - np. [logowanie](https://github.com/senko/python-logger/blob/master/logger.py)

## Metody specjalne
Metody zaczynające i kończące swoje nazwy od `__` to tzw. metody specjalne. W Pythonie istnieje wiele specjalnych metod, realizujacych określone funkcje (np. umożliwiające iterowanie po klasach, porównywanie etc.) jednak na tym szkoleniu skupimy się w szczególności na metodach `__init__`, `__str__`, `__repr__`, `__eq__` i `__hash__` - czyli tych najczęściej spotykanych w kodzie aplikacji.

In [ ]:
class Stock:
    def __init__(self, ticker, name):
        self.ticker = ticker
        self._company_name = name

    def __str__(self):
        return f"{self._company_name} stock"

    def __repr__(self):
        return f"Stock[ticker: {self.ticker}, company_name: {self._company_name}]"

    def __eq__(self, other):
        return other.ticker == self.tickere and other._company_name == self._company_name

    def __hash__(self):
        return hash((self.ticker, self._company_name))


### `__str__`
Ta metoda kontroluje w jaki sposób obiekty danej klasy są wypisywanie na ekran. Celem tej metody jest przedstawienie obiektu w sposób umożliwiający *ludziom* łatwe go odczytanie, inaczej mówiąc wygoda. 

### `__repr__`
Ta metoda kontroluje w jaki sposób obiekty danej klasy są wypisywanie na ekran, jednak jej celem nie jest czytelność dla `ludzi` a jednoznaczność i potencjalnie przetwarzanie automatyczne. Wykorzystuje się ją np. w debuggerach by nie mieć wątpliwości o który obiekt chodzi - `__str__` może być niejednoznaczny o ile poprawia to czytelność np.: `str("3") == str(3)`.

### `__eq__`
`__eq__` jest wywoływana przy porównaniu z użyciem `==` - dla obiektów domyślnie rozróżnienie następuje po referencji lub hashu, jeśli obiekt jest hashowalny. Przykładowo, mamy do zaimplementowania klasę `Money` - nie ma znaczenia, która konkretnie instancja jest w naszym posiadaniu - zamiast tego liczy się np. nominał i ilość egzemplarzy, które mamy. Aby to osiągnać nadpisujemy `__eq__` tak, by zwracała `True` zawsze gdy zgadza się nominał i ilość banknotów. 

### `__hash__`
Obiekty mogą być hashowalne lub nie - jeśli mają być, to metoda specjalna `__hash__` musi być sensownie zaimplemntowana. Po co przydaje się by obiekt był hashowalny? Np. po to by mógł być kluczem w słowniku lub elementem zbioru. Aby ułatwić nam implementacje istnieje wbudowana funkcja `hash`, której można podać dowolny hashowalny argument - np. `Tuple` zawierającą pola kontrybuujące do tego, że dwa egzemplarze klasy są rozróżnialne. Jeśli egzemplarze są równe w sensie operatora `==`, to powinny mieć ten sam hash (niekoniecznie w drugą stronę). Domyślnie hashowalne są typy proste i niezmienialne kolekcje - np. `Tuple` czy `frozenset`.

### Inne metody specjalne
Wiele wbudowanych zachowań różnorakich obiektów w Pythonie jest zaimplementowana przy użyciu metod specjalnych: przykładowo, iterowanie po kolekcjach jest możliwe dzięki temu, że implementują one metody `__iter__()` i `__next__()` - korzysta z nich nawet pętla `for`! Metodami specjalnymi są operatory (np.: porównanie przy użyciu `<` jest implementowane przez `__lt__()`, mnożenie jest operatorem `__mul__` etc.), odwołania do elementów i wiele innych mechanizmów. Nie będziemy ich wszystkich omawiać, ale warto wiedzieć jak tworzyć nasze własne klasy, które mogą zyskać na naturalności użycia i być bardziej ekspresywne.

### *Zadanie*
Zdefiniuj klasę typu mixin, której zadaniem będzie dodanie do klas po niej dziedziczących funkcjonalności zamiany ich obiektów na słowniki. Klasa powinna dodawać metodę `to_dict`, która będzie rekursywnie przechodzić po polach swoich podklas i każde z nich również próbować zamienić na słownik. Jeśli dane pole nie może być zamienione, jego wartość powinna bezpośrednio trafić do słownika obiektu, który go zawiera. Nazwy prywatne w Pythonie (zaczynające się na `__`) podlegają `mangling`-owi, czyli doklejana jest do nich nazwa klasy, prefiksowana przez `_`. W słowniku powinny jednak znaleźć się pod kluczem pozbawionym prefiksu '__' i nazwy klasy, np.: `__key_kpi` po manglowaniu staje się `_GrowthStock__key_kpi`, a w słowniku powinno być po prostu `key_kpi`. Przykładowo:
```
from decimal import Decimal

class Stock:
    def __init__(
        self: Self,
        company: Company,
        market_cap: Decimal,
        pe_ratio: float,
        eps: float,
        last_3_trades: list[Decimal]
    ):
        self.__company = company
        self.__market_cap = market_cap
        self.__pe_ratio = pe_ratio
        self.__eps = eps
        self.__last_3_trades = last_3_trades

class GrowthStock(DictConverterMixin, Stock):
    def __init__(
        self: Self,
        company: Company,
        market_cap: float,
        pe_ratio: float,
        eps: float,
        last_3_trades: list[Decimal],
        key_kpi: str
    ):
        self.__key_kpi
        super(DictConverterhMixin, self).__init__(company, market_cap, pe_ratio, eps, last_3_trades)

    @property
    def key_kpi(self):
        return self.__key_kpi

class Company(DictConverterMixin):
    def __init__(self, company_name: str, ticker: str):
        self.__ticker = ticker.upper()
        self.__name = company_name

    @property
    def ticker(self):
        return self.__ticker

    @property
    def name(self):
        return self.__name

snowflake = GrowthStock(
    Company("SNOW", "Snowflake"),
    market_cap=Decimal("5.468e10"),
    pe_ratio=float('NaN'),
    eps=-2.65,
    last_3_trades=[Decimal('165.68'), Decimal('166.02'), Decimal('165.34')],
    key_kpi="global cloud infrastructure spending")
snowflake.to_dict()
```

powinien wygenerować następujacy słownik:
```
{
    "company": {
        "ticker": "SNOW",
        "name": "Snowflake"
    },
    "market_cap": Decimal("5.468e10"),
    "pe_ratio": nan,
    "eps": -2.65,
    "last_3_trades": [Decimal('165.68'), Decimal('166.02'), Decimal('165.34')],
    "key_kpi": "global cloud infrastructure spending"
}
```
W pliku `examples/dict_mixin.py` znajduje się szkielet tej klasy i powyższy przykład, co powinno ułatwić testowanie.
Hint: Aby dostać się do atrybutów danego obiektu, można skorzystać z pola specjalnego `__dict__`.

## Dataclass
Często klasy są jedynie kontenerami na dane, bez rozbudowanych zachowań zmieniających ich wewnętrzny stan zgodnie z pewną logiką. Obiekty tych klas powinny umożliwiać łatwe tworzenie, porównywanie, wypisywanie w czytelny sposób swojej zawartości. Począwszy od Pythona 3.7 mamy do dyspozycji bibliotekę `dataclasses`, która upraszcza tworzenie i używanie takich klas - zamiast:

In [ ]:
class Card:
    def __init__(self, rank, suite):
        self.rank = rank
        self.suite = suite
    
    def __eq__(self, other):
        return isinstance(other, Card) and self.rank == other.rank and self.suite == other.suite
    
    def __repr__(self):
        return f"Card({self.rank} {self.suite})"

queen_of_hearts = Card('Q', '♡')
another_queen_of_hearts = Card('Q', '♡')
assert queen_of_hearts == another_queen_of_hearts

... możemy zrobić jedynie tyle:

In [ ]:
from dataclasses import dataclass

@dataclass
class Card:
    rank: str
    suite: str


queen_of_hearts = Card('Q', '♡')
another_queen_of_hearts = Card('Q', '♡')
assert queen_of_hearts == another_queen_of_hearts

Jak widać, `dataclasses` oszczędza nam sporo pisania - wystarczy, że odpowiednie pola zostaną zadeklarowane wraz ze swoim typem (konieczne! Ale może być to `Any`), a resztę zrobi za nas biblioteka. Przydatne bywa też uczynienie naszej klasy niezmienialną (*ang. mutable*) - nic prostszego! `dataclasses` czyni to naprawdę prostym:

In [ ]:
from dataclasses import dataclass

@dataclass
class Card:
    rank: str
    suite: str

queen_of_hearts = Card('Q', '♡')
queen_of_hearts.rank = 'K'


@dataclass(frozen=True)
class ImmutableCard:
    rank: str
    suite: str

immutable_queen_of_hearts = ImmutableCard('Q', '♡')
immutable_queen_of_hearts.rank = 'K'

Oczywiście, pola takich klas mogą mieć wartości domyślne:

In [ ]:
from dataclasses import dataclass
@dataclass
class Card:
    rank: str = get_rank()
    suite: str = '♣'

Card()

Nie należy jednak bezpośrednio wyliczać ich wywołaniem funkcji! Jest to częsty błąd w Pythonie, polegający na użyciu mutowalnych argumentów domyślnych - logika wewnątrz `dataclasses` pod spodem wygeneruje normalną klasę, zawierającą np. konstruktor (`__init__`), który dostanie podane tu wartości jako parametry. Jeśli chcemy dynamicznie lub leniwie produkować wartości domyślne możemy to zrobić tak:

In [ ]:
from dataclasses import dataclass, field
import random

RANKS = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
SUITS = '♣ ♢ ♡ ♠'.split()

@dataclass
class Card:
    rank: str = field(default_factory=lambda: random.choice(RANKS))
    suite: str = '♣'

Card()

`field` może zmieniać różne inne parametry pola - domyślną wartość (stałą), sposób reprezentacji, porównywania, liczenia hashu etc.

**UWAGA**

In [ ]:
# tu parametr domyślny jest wyliczany tylko raz!!!
def fun1(x, lst = []):
    lst.append(x)
    return lst


print(fun1(5))
print(fun1(6))

# fix
def fun2(x, lst = None):
    if not lst:
        lst = []
    lst.append(x)
    return lst

# print(fun2(5))
# print(fun2(6))

#### Dataclasses vs namedtuple
Historycznie podobną rolę do Dataclass pełniły tzw. nazwane krotki - *namedtuples*. Tworzy się je tak:

In [ ]:
from collections import namedtuple

# Definiowanie namedtuple
Osoba = namedtuple('Osoba', ['imie', 'nazwisko', 'wiek'])

# Tworzenie instancji
jan = Osoba(imie='Jan', nazwisko='Kowalski', wiek=30)

print(jan.imie)  # Wydrukuje: Jan
print(jan[0])    # Również wydrukuje: Jan

Zalety namedtuples:
-  niezmienne (immutable), co oznacza, że ich wartości nie mogą być zmieniane po utworzeniu.
- Są mniej kosztowne pod względem pamięci niż regularne klasy.
- Dają możliwość dostępu do pól zarówno przez nazwę, jak i indeks.

Dataclasses są jednak zdecydowanie bardziej elastyczne, toteż większość projektów opartych na Pythonie 3.7 i nowszym używa Dataclass zamiast namedtuples. 

#### Dataclassess vs attrs
Attrs to biblioteka zewnętrzna, która również pozwala na definiowanie klas z automatycznym generowaniem specjalnych metod, podobnie jak dataclasses, ale jest dostępna również dla starszych wersji Pythona i oferuje nieco szerszą funkcjonalność, zwłaszcza w zakresie walidacji danych. Standardowa biblioteka dataclasses była tak naprawdę zaimplementowana na wzór attr i jest względem niej intencjonalnie okrojona. Jest natomiast dostępna w każdej instalacji (C)Pythona i używa nieco krótszej składni.

### *Zadanie*
Uprość następujący kod, dbając o jego czytelność i intuicyjność użycia. Zadbaj by po modyfikacjach również nie dało się (łatwo) modyfikować pól obiektów tej klasy po ich utworzeniu. Zachowaj wartości domyślne.

In [1]:
from decimal import Decimal

def fetch_pe_ratio(ticker):
    return 25.0

class Stock:
    def __init__(self, ticker, market_cap, pe_ratio, eps, last_3_trades):
        self.__ticker = ticker
        self.__market_cap = market_cap
        self.__pe_ratio = pe_ratio
        self.__eps = eps
        self.__last_3_trades = last_3_trades

    def __eq__(self, other):
        is_eq = self.__ticker == other.__ticker
        is_eq &= self.__market_cap == other.__market_cap
        is_eq &= self.__pe_ratio == other.__pe_ratio
        is_eq &= self.__eps == other.__eps
        return is_eq

    def __hash__(self):
        return hash((self.__ticker, self.__market_cap, self.__pe_ratio, self.__eps))

    @property
    def ticker(self):
        return self._ticker

    @property
    def market_cap(self):
        return self.__market_cap

    @property
    def pe_ratio(self):
        return self.__pe_ratio

    @property
    def eps(self):
        return self.__eps

    @property
    def last_3_trades(self):
        return self.__last_3_trades

    class Builder:
        def __init__(self):
            self.__stock_dict = {}

        def with_ticker(self, ticker):
            self.__stock_dict["ticker"] = ticker
            return self

        def with_market_cap(self, market_cap):
            self.__stock_dict["market_cap"] = market_cap
            return self

        def with_pe_ratio(self, pe_ratio):
            self.__stock_dict["pe_ratio"] = pe_ratio
            return self

        def with_eps(self, eps):
            self.__stock_dict["eps"] = eps
            return self

        def with_last_3_trades(self, last_3_trades):
            self.__stock_dict["last_3_trades"] = last_3_trades
            return self

        def build(self):
            if "ticker" not in self.__stock_dict:
                raise TypeError("ticker must be specified")
            if "market_cap" not in self.__stock_dict:
                self.__stock_dict["market_cap"] = "UNKNOWN"
            if "pe_ratio" not in self.__stock_dict:
                self.__stock_dict["pe_ratio"] = fetch_pe_ratio()
            if "eps" not in self.__stock_dict:
                self.__stock_dict["eps"] = float(input(f"provide EPS: "))
            if "last_3_trades" not in self.__stock_dict:
                self.__stock_dict["last_3_trades"] = (100,200,300)
            return Stock(**self.__stock_dict)


stock1 = Stock.Builder()\
.with_ticker("NVDA")\
.with_market_cap("1.2T")\
.with_eps(4.18)\
.with_pe_ratio(116.32)\
.build()

stock2 = Stock.Builder()\
.with_ticker("NVDA")\
.with_market_cap("1.2T")\
.with_eps(Decimal('4.18'))\
.with_pe_ratio(Decimal('116.32'))\
.build()

stock1 == stock2
hash(stock1)

8073665065068212382